# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('Weather_Data.csv')

weather_data.head()

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country
0,0,Jumla,29.2747,82.1838,286.03,99,100,1.64,NP
1,1,Hobart,-42.8794,147.3294,283.08,63,75,0.45,AU
2,2,Bowen,-20.0167,148.2333,300.15,74,8,5.07,AU
3,3,Albany,42.6001,-73.9662,280.23,77,50,0.89,US
4,4,Pacifica,37.6138,-122.4869,288.79,72,40,1.79,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(g_key)

In [5]:
location = weather_data[["Latitude", "Longitude"]]
humid = weather_data['Humidity']

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(location, weights=humid,
                                dissipating=False, max_intensity = 100, point_radius=1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
perfect_weather = weather_data.loc[(weather_data['Humidity'] < 35) & (weather_data['Temperature'] > 275)
                                   & (weather_data['Windspeed'] < 2), :]

perfect_weather = perfect_weather.dropna(how='any')
perfect_weather.reset_index(inplace=True, drop=True)
perfect_weather

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness,Windspeed,Country
0,19,Gazalkent,41.5581,69.7708,280.35,33,74,1.78,UZ
1,57,Mar del Plata,-38.0023,-57.5575,286.26,21,0,1.34,AR
2,76,Atar,20.5169,-13.0499,305.12,11,100,1.39,MR
3,93,Nizwá,22.9333,57.5333,299.54,24,4,0.79,OM
4,117,Jiuquan,39.7432,98.5174,279.15,29,100,1.28,CN
5,161,Ponta do Sol,32.6667,-17.1000,295.68,25,88,1.44,PT
6,170,Tabas,33.5959,56.9244,294.02,17,0,0.32,IR
7,237,Najrān,17.4924,44.1277,292.51,23,4,1.09,SA
8,242,Aksu,41.1231,80.2644,284.98,23,100,1.15,CN
9,283,Udaipur,24.5712,73.6918,294.07,27,2,1.91,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = perfect_weather[['City', "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""

In [8]:
hotel_name = []

for index, row in hotel_df.iterrows():
    Lat = row['Latitude']
    Long = row['Longitude']
    city = row['City']

    # geocoordinates
    target_coordinates = f"{Lat},{Long}"
    target_radius = 5000
    target_type = "hotel"

    # set up a parameters dictionary
    params = {
        "location": target_coordinates,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # run a request using our params dictionary
    response = requests.get(base_url, params=params).json()
    #print(target_coordinates)
    try:
        hotel_df.loc[index, "Hotel Name"] = response['results'][0]['name']
        print(f'Loaded data for hotel name')
    except:
        print(f'Hotel not found, skipped')
        pass

hotel_df

Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name
Loaded data for hotel name


,City,Country,Latitude,Longitude,Hotel Name
0,Gazalkent,UZ,41.5581,69.7708,Gazalkent
1,Mar del Plata,AR,-38.0023,-57.5575,Mar del Plata
2,Atar,MR,20.5169,-13.0499,Atar
3,Nizwá,OM,22.9333,57.5333,Nizwa
4,Jiuquan,CN,39.7432,98.5174,Jiuquan
5,Ponta do Sol,PT,32.6667,-17.1000,Ribeira Brava
6,Tabas,IR,33.5959,56.9244,Tabas
7,Najrān,SA,17.4924,44.1277,Najran
8,Aksu,CN,41.1231,80.2644,Aksu Prefecture
9,Udaipur,IN,24.5712,73.6918,Udaipur


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [18]:
# Add marker layer ontop of heat map
marker = gmaps.marker_layer(locations, hover_text=hotel_info)
fig.add_layer(marker)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))